In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
"""
import kagglehub
kagglehub.login()
"""

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
"""
playground_series_s5e7_path = kagglehub.competition_download('playground-series-s5e7')

print('Data source import complete.')
"""

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 시각화
import matplotlib.pyplot as plt

# 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

#하이퍼 파라미터 튜닝: 보통 랜덤서치로 대략 파악한 다음 그리드 서치로 미세조정
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# 모델
from xgboost import XGBClassifier

# 평가도구
from sklearn.metrics import accuracy_score

# 분포
from scipy.stats import uniform, loguniform, randint

In [2]:
train = pd.read_csv('./train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [3]:
test = pd.read_csv('./test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175 entries, 0 to 6174
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         6175 non-null   int64  
 1   Time_spent_Alone           5750 non-null   float64
 2   Stage_fear                 5577 non-null   object 
 3   Social_event_attendance    5778 non-null   float64
 4   Going_outside              5709 non-null   float64
 5   Drained_after_socializing  5743 non-null   object 
 6   Friends_circle_size        5825 non-null   float64
 7   Post_frequency             5767 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 386.1+ KB


In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train['Personality'].value_counts()

In [7]:
# 범주형 변수 변환
for df in [train, test]:
    df['Stage_fear'] = df['Stage_fear'].map({'No': 0, 'Yes': 1})
    df['Drained_after_socializing'] = df['Drained_after_socializing'].map({'No': 0, 'Yes': 1})

In [9]:
train['Stage_fear'].unique()

array([ 0.,  1., nan])

In [10]:
# 라벨 변환
train['Personality'] = train['Personality'].map({'Introvert': 0, 'Extrovert': 1})

In [11]:
train['Personality'].unique()

array([1, 0], dtype=int64)

In [12]:
# 데이터 분리
X = train.drop(columns=['id', 'Personality'])
y = train['Personality']
X_test = test.drop(columns=['id'])

In [24]:
# XGBoost 모델 + 하이퍼파라미터 탐색
param_dist = {
    'n_estimators': [300, 500, 800, 1000],
    'max_depth': [3, 4, 5, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# RandomizedSearchCV
xgb = XGBClassifier(random_state=42)
random_search = RandomizedSearchCV(  # 교차 검증 내부진행 : 데이터분할X
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # 시도 횟수 (높이면 더 정밀)
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X, y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constrain...
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None, ...),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0, 1, 3],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 8, 10],
                                        'n_estimators': [300, 500, 800, 1000],
                                        'reg_alpha': [0, 0.01, 0.1],
                                        'reg_lambda': [1, 1.5, 2],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='accuracy', verbose=2)

In [25]:
random_search.score(X,y)

0.9693910602461672

In [72]:
# XGBoost 모델 + 하이퍼파라미터 탐색
param_dist = {
    'n_estimators': randint(300,1000),
    'max_depth': randint(3,10),
    'learning_rate': uniform(0.01,0.2),
    'subsample': uniform(0.6,1.0),
    'colsample_bytree': uniform(0.6, 1.0),
    'gamma': randint(0,3),
    'reg_alpha': loguniform(0.00001,0.1),
    'reg_lambda': uniform(1,2)
}

# RandomizedSearchCV
xgb = XGBClassifier(random_state=123)
random_search3 = RandomizedSearchCV(  # 교차 검증 내부진행 : 데이터분할X
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # 시도 횟수 (높이면 더 정밀)
    scoring='accuracy',
    cv=3,
    verbose=2,
    random_state=123,
    n_jobs=-1
)

random_search3.fit(X, y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\itwill\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
63 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\itwill\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\itwill\anaconda3\Lib\site-packages\xgboost\core.py", line 729, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\itwill\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1682, in fit
    self._Booster = train(
                    ^^^^^^
  File "C:\Users\itwi

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constrain...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020CDA7A9370>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CDFD9E420>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CDBAE0680>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CDFD9CB60>},
                   random_state=123, scoring='accuracy', verbose=2)

In [60]:
random_search2.score(X,y)

0.9700928525156554

In [61]:
random_search2.best_params_

{'colsample_bytree': 0.6890516974824745,
 'gamma': 1,
 'learning_rate': 0.06272576573201136,
 'max_depth': 9,
 'n_estimators': 815,
 'reg_alpha': 0.0032167236409001558,
 'reg_lambda': 2.114899621144565,
 'subsample': 0.918579559579802}

In [26]:
# 최적 모델
best_params = random_search.best_params_
print("Best Params:", best_params)

final_model = XGBClassifier(**best_params, random_state=42)
final_model.fit(X, y)  # 전체 데이터로 다시 학습

Best Params: {'subsample': 0.6, 'reg_lambda': 1.5, 'reg_alpha': 0.1, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 3, 'colsample_bytree': 0.6}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=3, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, ...)

In [27]:
final_model.score(X,y)

0.9693910602461672

In [28]:
# 테스트 파일 예측
preds = final_model.predict(X_test)
preds_labels = ['Extrovert' if x == 1 else 'Introvert' for x in preds]

In [587]:
# 제출 파일 생성
submission = pd.DataFrame({'id': test['id'], 'Personality': preds_labels})
submission.to_csv('preds_labels.csv', index=False)

In [66]:
preds2 = random_search2.predict(X_test)
preds_labels2 = ['Extrovert' if x == 1 else 'Introvert' for x in preds2]

In [39]:
p1=pd.Series(preds_labels)
p1

0       Extrovert
1       Introvert
2       Extrovert
3       Extrovert
4       Introvert
          ...    
6170    Extrovert
6171    Introvert
6172    Extrovert
6173    Extrovert
6174    Introvert
Length: 6175, dtype: object

In [67]:
p2=pd.Series(preds_labels2)
p2

0       Extrovert
1       Introvert
2       Extrovert
3       Extrovert
4       Introvert
          ...    
6170    Extrovert
6171    Introvert
6172    Extrovert
6173    Extrovert
6174    Introvert
Length: 6175, dtype: object

In [68]:
sum(p1!=p2)

5

In [182]:
5/6174, 7/6174

(0.0008098477486232588, 0.0011337868480725624)

In [586]:
# 제출 파일 생성
submission = pd.DataFrame({'id': test['id'], 'Personality': preds_labels2})
submission.to_csv('./preds_labels2.csv', index=False)

In [183]:
0.975708-0.0008098477486232588, 0.975708+0.0008098477486232588, 0.975708+0.0011337868480725624

(0.9748981522513768, 0.9765178477486233, 0.9768417868480725)

In [598]:
1/6174

0.00016196954972465177

In [600]:
0.975708+0.00016196954972465177

0.9758699695497247

In [512]:
# XGBoost 모델 + 하이퍼파라미터 탐색
param_dist = {
    'n_estimators': randint(100, 300),            # 100 ~ 299
    'max_depth': randint(3, 10),                  # 3 ~ 9
    'learning_rate': uniform(0.01, 0.2),          # 0.01 ~ 0.21
    'subsample': uniform(0.4, 0.6),               # ✅ 0.4 ~ 1.0
    'colsample_bytree': uniform(0.4, 0.6),        # ✅ 0.4 ~ 1.0
    'gamma': randint(0, 3),                       # 0, 1, 2
    'reg_alpha': loguniform(1e-5, 0.1),           # 1e-5 ~ 0.1 (log scale)
    'reg_lambda': uniform(1, 1)                   # ✅ 1.0 ~ 2.0
}

# RandomizedSearchCV
xgb = XGBClassifier()
random_search3 = RandomizedSearchCV(  # 교차 검증 내부진행 : 데이터분할X
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=1,  # 시도 횟수 (높이면 더 정밀)
    scoring='accuracy',
    cv=2,
    verbose=2,
    n_jobs=-1
)

random_search3.fit(X, y)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           feature_weights=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constrain...
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020CE1DC9040>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE2F19640>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE2F1A060>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE2F19670>},
                   scoring='accuracy', verbose=2)

In [513]:
random_search3.score(X,y)

0.9729000215936083

In [514]:
preds3 = random_search3.predict(X_test)
preds_labels3 = ['Extrovert' if x == 1 else 'Introvert' for x in preds3]

In [515]:
p3=pd.Series(preds_labels3)
p3

0       Extrovert
1       Introvert
2       Extrovert
3       Extrovert
4       Introvert
          ...    
6170    Extrovert
6171    Introvert
6172    Extrovert
6173    Extrovert
6174    Introvert
Length: 6175, dtype: object

In [516]:
sum(p1!=p3)

16

In [517]:
sum(p2!=p3)

15

In [521]:
accuracy_score(preds_labels,preds_labels3), accuracy_score(preds_labels2,preds_labels3)

(0.9974089068825911, 0.9975708502024292)

In [524]:
from lightgbm import LGBMClassifier

In [580]:
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),         # 0.01 ~ 0.21
    'subsample': uniform(0.4, 0.6),              # 0.4 ~ 1.0
    'colsample_bytree': uniform(0.4, 0.6),       # 0.4 ~ 1.0
    'reg_alpha': loguniform(1e-5, 0.1),          # L1 규제
    'reg_lambda': uniform(1.0, 1.0),             # 1.0 ~ 2.0
    'min_child_samples': randint(10, 50),
    'num_leaves': randint(20, 150)
}

lgb_model = LGBMClassifier()

random_search3 = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_dist,
    n_iter=1,                # 탐색 횟수
    scoring='accuracy',       # 또는 'roc_auc', 'f1', 'neg_log_loss'
    cv=2,
    verbose=1,
    n_jobs=-1
)
random_search3.fit(X, y)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[LightGBM] [Info] Number of positive: 13699, number of negative: 4825
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000343 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 69
[LightGBM] [Info] Number of data points in the train set: 18524, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.739527 -> initscore=1.043512
[LightGBM] [Info] Start training from score 1.043512
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

RandomizedSearchCV(cv=2, estimator=LGBMClassifier(), n_iter=1, n_jobs=-1,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE63B1640>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE63B0890>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete...
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020CE63B17C0>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE63B1820>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE63B1A30>,
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020CE63B0B30>},
                   scoring='accuracy', verbose=1)

In [581]:
random_search3.score(X,y)

0.9716044050960916

In [582]:
preds3 = random_search3.predict(X_test)
preds_labels3 = ['Extrovert' if x == 1 else 'Introvert' for x in preds3]

In [583]:
p3=pd.Series(preds_labels3)
p3

0       Extrovert
1       Introvert
2       Extrovert
3       Extrovert
4       Introvert
          ...    
6170    Extrovert
6171    Introvert
6172    Extrovert
6173    Extrovert
6174    Introvert
Length: 6175, dtype: object

In [594]:
sum(p1!=p3), sum(p2!=p3)

(5, 8)

In [585]:

submission = pd.DataFrame({'id': test['id'], 'Personality': preds_labels3})
submission.to_csv('./submission_lgbm5.csv', index=False)


In [590]:
p1[p1!=p3]

958     Introvert
1088    Introvert
1493    Extrovert
4806    Extrovert
5481    Extrovert
dtype: object

In [591]:
p3[p1!=p3]

958     Extrovert
1088    Extrovert
1493    Introvert
4806    Introvert
5481    Introvert
dtype: object

In [593]:
test.iloc[[958,1088,1493,4806,5481],:]

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
958,19482,10.0,0.0,0.0,0.0,1.0,1.0,2.0
1088,19612,5.0,0.0,0.0,0.0,1.0,4.0,3.0
1493,20017,0.0,1.0,3.0,3.0,1.0,5.0,3.0
4806,23330,5.0,0.0,9.0,7.0,0.0,12.0,5.0
5481,24005,9.0,0.0,9.0,7.0,0.0,12.0,10.0


In [ ]:
i, i ,e, e, e 